In [43]:
import nibabel as nib
#from monai.transforms import Compose, LoadImaged, AddChanneld, ToTensord
from monai.inferers import sliding_window_inference
from monai.networks.nets import UNet
from monai.data import Dataset, DataLoader
import torch
import nibabel as nib
import numpy as np
import pandas as pd
import os
import glob

In [44]:
combined_roi_dict = {
    'Left-Cingulate': {
        'ctx-lh-caudalanteriorcingulate',
        'ctx-lh-rostralanteriorcingulate',
        'ctx-lh-isthmuscingulate',
        'ctx-lh-posteriorcingulate',
    },
    'Right-Cingulate': {
        'ctx-rh-caudalanteriorcingulate',
        'ctx-rh-rostralanteriorcingulate',
        'ctx-rh-isthmuscingulate',
        'ctx-rh-posteriorcingulate',
    },
    'Left-Frontal': {
        'ctx-lh-caudalmiddlefrontal',
        'ctx-lh-lateralorbitofrontal',
        'ctx-lh-medialorbitofrontal',
        'ctx-lh-parsopercularis',
        'ctx-lh-parsorbitalis',
        'ctx-lh-parstriangularis',
        'ctx-lh-precentral',
        'ctx-lh-rostralmiddlefrontal',
        'ctx-lh-superiorfrontal',
        'ctx-lh-paracentral'
    },
    'Right-Frontal': {
        'ctx-rh-caudalmiddlefrontal',
        'ctx-rh-lateralorbitofrontal',
        'ctx-rh-medialorbitofrontal',
        'ctx-rh-parsopercularis',
        'ctx-rh-parsorbitalis',
        'ctx-rh-parstriangularis',
        'ctx-rh-precentral',
        'ctx-rh-rostralmiddlefrontal',
        'ctx-rh-superiorfrontal',
        'ctx-rh-paracentral'
    },
    'Left-Temporal': {
        'ctx-lh-entorhinal',
        'ctx-lh-fusiform',
        'ctx-lh-inferiortemporal',
        'ctx-lh-middletemporal',
        'ctx-lh-superiortemporal',
        'ctx-lh-transversetemporal',
        'ctx-lh-parahippocampal',

    },
    'Right-Temporal': {
        'ctx-rh-entorhinal',
        'ctx-rh-fusiform',
        'ctx-rh-inferiortemporal',
        'ctx-rh-middletemporal',
        'ctx-rh-superiortemporal',
        'ctx-rh-transversetemporal',
        'ctx-rh-parahippocampal',
       
    },
    'Left-Parietal': {
        'ctx-lh-inferiorparietal',
        'ctx-lh-postcentral',
        'ctx-lh-superiorparietal',
        'ctx-lh-supramarginal',
        'ctx-lh-precuneus'
    },
    'Right-Parietal': {
        'ctx-rh-inferiorparietal',
        'ctx-rh-postcentral',
        'ctx-rh-superiorparietal',
        'ctx-rh-supramarginal',
        'ctx-rh-precuneus'
    },
    'Left-Occipital': {
        'ctx-lh-cuneus',
        'ctx-lh-lateraloccipital',
        'ctx-lh-lingual',
        'ctx-lh-pericalcarine'
    },
    'Right-Occipital': {
        'ctx-rh-cuneus',
        'ctx-rh-lateraloccipital',
        'ctx-rh-lingual',
        'ctx-rh-pericalcarine'
    },
    'Left-Insula':{
        'ctx-lh-insula'
    },
    'Right-Insula': {
        'ctx-rh-insula'
    },
    'Left-Cerebellum':{
        'Left-Cerebellum-White-Matter',
        'Left-Cerebellum-Cortex'
    },
    'Right-Cerebellum': {
        'Right-Cerebellum-White-Matter',
        'Right-Cerebellum-Cortex'
    },
    'Left-Combined-Cerebral-WM': {
        'Left-Cerebral-White-Matter',
        'Left-WM-hypointensities'
    },
    'Right-Combined-Cerebral-WM': {
        'Right-Cerebral-White-Matter',
        'Right-WM-hypointensities'
    },
    'Left-Total-Lateral-Ventricle': {
        'Left-Lateral-Ventricle',
       # 'Left-Inf-Lat-Vent'
    },
    'Right-Total-Lateral-Ventricle': {
        'Right-Lateral-Ventricle',
        #'Right-Inf-Lat-Vent'
    },
}

In [45]:
def load_pickle_file(filepath):
    with open(filepath, 'rb') as file:
        data = pickle.load(file)
    return data
# 파일열어보기 , PICKLE로 되어있는것

In [46]:

base_dir = '/mnt/drwelly1/Team_Planning/Brain_Volumetry/OASIS1_average_after'


def calculate_volume(nii_path):
    img = nib.load(nii_path) 
    data = img.get_fdata()
    header = img.header
    voxel_volume = np.prod(header.get_zooms())
    unique_elements, counts_elements = np.unique(data, return_counts=True)
    volumes = counts_elements * voxel_volume
    return dict(zip(unique_elements.astype(int), volumes))

In [47]:
import os
import pandas as pd



volume_data = []

for subject_dir in os.listdir(base_dir):
    full_subject_dir = os.path.join(base_dir, subject_dir)
    if not os.path.isdir(full_subject_dir) or 'FS222' not in subject_dir:
        continue
    
    double_subject_dir = os.path.join(full_subject_dir, subject_dir)
    if not os.path.isdir(double_subject_dir):
        continue
    
    mri_dir = os.path.join(double_subject_dir, 'mri')
    nii_path = os.path.join(mri_dir, 'aparc.DKTatlas+aseg.deep.nii.gz')
    
    if os.path.isfile(nii_path):
        try:
            vol_data = calculate_volume(nii_path)
            vol_data["subject"] = subject_dir
            volume_data.append(vol_data)
        except OSError as e:
            print(f"Error processing file {nii_path}, skipped  Error: {e}")
    else:
        print(f"File not found, skipped: {nii_path}")

volume_df = pd.DataFrame(volume_data)

KeyboardInterrupt: 

In [15]:
volume_df.shape

(436, 97)

In [16]:
volume_df['subject'] = volume_df['subject'].str.replace('_Tag_pr100_sr100_ppf100_spf100_ipf100_sipf143_FS222', '')


In [17]:
volume_df

,0,2,4,5,7,8,10,11,12,13,...,2025,2026,2027,2028,2029,2030,2031,2034,2035,subject
0,1.584253e+07,175160.836321,32425.847504,1823.044972,9411.233856,40704.152378,5705.118913,3515.406946,3861.493941,1582.258976,...,7395.079887,1369.941979,8292.367873,22628.738654,8267.328874,12038.270816,7424.920886,779.981988,4928.566925,OAS1_0299_MR1
1,1.553299e+07,245307.765249,6128.037906,227.408997,14179.962783,60794.691070,8367.827872,4056.660938,6167.482906,2305.645965,...,11983.390817,3060.588953,14439.270779,31286.058522,11512.794824,16577.532746,9696.266852,1071.188984,6528.661900,OAS1_0131_MR1
2,1.594658e+07,168228.463427,5420.428917,238.727996,10140.794845,42036.707357,5738.046912,2735.767958,3753.105943,1387.434979,...,6626.416899,1950.983970,9314.507858,20860.916681,8295.111873,12167.924814,6636.706899,801.933988,4644.905929,OAS1_0355_MR1
3,1.588463e+07,170992.014385,4308.079934,133.769998,10988.690832,45446.470305,6529.004900,2982.041954,4208.609936,1649.486975,...,9641.729853,1947.210970,9966.207848,21957.487664,9939.453848,13753.956790,9475.374855,855.784987,4675.089929,OAS1_0387_MR1
4,1.590237e+07,175719.926313,31337.165521,1242.345981,10147.997845,37096.135433,6087.220907,3054.757953,5442.723917,1690.303974,...,6705.649897,2039.134969,5881.763910,16938.025741,6865.830895,10369.232841,6915.222894,433.551993,5155.975921,OAS1_0380_MR1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,1.569422e+07,213351.484737,6103.684907,324.134995,11975.158817,51164.623218,7714.412882,3909.513940,4393.486933,1831.276972,...,10200.819844,2284.379965,13574.910792,24981.718618,10285.883843,14791.874774,9542.259854,743.623989,4902.155925,OAS1_0104_MR1
432,1.538854e+07,278761.240737,8439.857871,297.037995,15907.310757,70669.317919,9776.871850,4169.164936,5788.810911,2158.498967,...,12471.822809,2771.782958,15401.042764,37759.497423,13282.674797,17660.383730,12286.945812,1197.069982,6543.410900,OAS1_0108_MR1
433,1.577001e+07,201174.298924,26875.078589,1254.350981,11974.815817,48770.483254,6626.073899,3760.994942,4147.555937,1796.976973,...,8750.272866,1626.162975,8410.359871,22483.306656,8038.204877,12119.561815,7643.068883,835.547987,5586.783915,OAS1_0425_MR1
434,1.586539e+07,186564.900147,29671.214546,485.687993,11138.238830,41450.863366,5801.158911,2985.471954,3590.866945,1601.809976,...,7590.589884,2158.498967,7412.229887,19157.921707,8613.415868,10714.290836,7784.384881,769.005988,5525.729915,OAS1_0035_MR1


In [18]:
excel_path = '/mnt/drwelly1/SeoyoungLim/Oasis_analysis/DrWelly_Volumetry_Colorcode.xlsx'

In [19]:
def load_excel_as_lut(excel_path):
    # Load the Excel file into a pandas DataFrame
    df = pd.read_excel(excel_path)
    
    # Create a dictionary to hold your LUT
    lut = {}
    
    # Iterate over the rows of the DataFrame and populate the dictionary
    for index, row in df.iterrows():
        # Assuming the ID is in the first column and the label in the second
        lut[row[0]] = row[1]
    
    return lut

freesurfer_lut= load_excel_as_lut(excel_path)

/tmp/ipykernel_3706191/2193877719.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lut[row[0]] = row[1]


In [20]:
# 모든 컬럼 이름을 변경
new_columns = [freesurfer_lut.get(int(col), 'Unknown') if col != 'subject' else 'subject' for col in volume_df.columns]


volume_df.columns = new_columns

In [21]:
volume_df

,Unknown,Left-Cerebral-White-Matter,Left-Lateral-Ventricle,Left-Inf-Lat-Vent,Left-Cerebellum-White-Matter,Left-Cerebellum-Cortex,Left-Thalamus,Left-Caudate,Left-Putamen,Left-Pallidum,...,ctx-rh-precuneus,ctx-rh-rostralanteriorcingulate,ctx-rh-rostralmiddlefrontal,ctx-rh-superiorfrontal,ctx-rh-superiorparietal,ctx-rh-superiortemporal,ctx-rh-supramarginal,ctx-rh-transversetemporal,ctx-rh-insula,subject
0,1.584253e+07,175160.836321,32425.847504,1823.044972,9411.233856,40704.152378,5705.118913,3515.406946,3861.493941,1582.258976,...,7395.079887,1369.941979,8292.367873,22628.738654,8267.328874,12038.270816,7424.920886,779.981988,4928.566925,OAS1_0299_MR1
1,1.553299e+07,245307.765249,6128.037906,227.408997,14179.962783,60794.691070,8367.827872,4056.660938,6167.482906,2305.645965,...,11983.390817,3060.588953,14439.270779,31286.058522,11512.794824,16577.532746,9696.266852,1071.188984,6528.661900,OAS1_0131_MR1
2,1.594658e+07,168228.463427,5420.428917,238.727996,10140.794845,42036.707357,5738.046912,2735.767958,3753.105943,1387.434979,...,6626.416899,1950.983970,9314.507858,20860.916681,8295.111873,12167.924814,6636.706899,801.933988,4644.905929,OAS1_0355_MR1
3,1.588463e+07,170992.014385,4308.079934,133.769998,10988.690832,45446.470305,6529.004900,2982.041954,4208.609936,1649.486975,...,9641.729853,1947.210970,9966.207848,21957.487664,9939.453848,13753.956790,9475.374855,855.784987,4675.089929,OAS1_0387_MR1
4,1.590237e+07,175719.926313,31337.165521,1242.345981,10147.997845,37096.135433,6087.220907,3054.757953,5442.723917,1690.303974,...,6705.649897,2039.134969,5881.763910,16938.025741,6865.830895,10369.232841,6915.222894,433.551993,5155.975921,OAS1_0380_MR1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,1.569422e+07,213351.484737,6103.684907,324.134995,11975.158817,51164.623218,7714.412882,3909.513940,4393.486933,1831.276972,...,10200.819844,2284.379965,13574.910792,24981.718618,10285.883843,14791.874774,9542.259854,743.623989,4902.155925,OAS1_0104_MR1
432,1.538854e+07,278761.240737,8439.857871,297.037995,15907.310757,70669.317919,9776.871850,4169.164936,5788.810911,2158.498967,...,12471.822809,2771.782958,15401.042764,37759.497423,13282.674797,17660.383730,12286.945812,1197.069982,6543.410900,OAS1_0108_MR1
433,1.577001e+07,201174.298924,26875.078589,1254.350981,11974.815817,48770.483254,6626.073899,3760.994942,4147.555937,1796.976973,...,8750.272866,1626.162975,8410.359871,22483.306656,8038.204877,12119.561815,7643.068883,835.547987,5586.783915,OAS1_0425_MR1
434,1.586539e+07,186564.900147,29671.214546,485.687993,11138.238830,41450.863366,5801.158911,2985.471954,3590.866945,1601.809976,...,7590.589884,2158.498967,7412.229887,19157.921707,8613.415868,10714.290836,7784.384881,769.005988,5525.729915,OAS1_0035_MR1


In [48]:
def update_df_columns_keep_subject(df, roi_dict):
    
    columns_to_remove = []
    
   
    protected_columns = ['subject','Left-Hippocampus','Right-Hippocampus', 'Left-Amygdala','Right-Amygdala', 'Left-Inf-Lat-Vent','Right-Inf-Lat-Vent'
                        ,'Left-Lateral-Ventricle','Right-Lateral-Ventricle']
                         

                         
    for roi, sub_columns in roi_dict.items():
      
        relevant_columns = [col for col in df.columns if col.split(' ')[0] in sub_columns and col not in protected_columns]
        
    
        if relevant_columns:
         
            df[roi] = df[relevant_columns].fillna(0).sum(axis=1)
          
            columns_to_remove.extend(relevant_columns)
 
    df = df.drop(columns=[col for col in columns_to_remove if col not in protected_columns], errors='ignore')
    
    
    return df

In [49]:


new_df1 = update_df_columns_keep_subject(volume_df.copy(), combined_roi_dict)

In [50]:
new_df1 

,Unknown,Left-Lateral-Ventricle,Left-Inf-Lat-Vent,Left-Thalamus,Left-Caudate,Left-Putamen,Left-Pallidum,3rd-Ventricle,4th-Ventricle,Brain-Stem,...,Left-Parietal,Right-Parietal,Left-Occipital,Right-Occipital,Left-Insula,Right-Insula,Left-Cerebellum,Right-Cerebellum,Left-Combined-Cerebral-WM,Right-Combined-Cerebral-WM
0,1.584253e+07,32425.847504,1823.044972,5705.118913,3515.406946,3861.493941,1582.258976,2313.191965,3629.625944,16986.731740,...,42131.375356,42606.773348,19008.030709,18849.907712,4737.858928,4928.566925,50115.386234,50087.946234,175160.836321,173277.423350
1,1.553299e+07,6128.037906,227.408997,8367.827872,4056.660938,6167.482906,2305.645965,1145.962982,1914.625971,21888.544665,...,58209.843110,59734.478087,29292.199552,30435.075535,6795.172896,6528.661900,74974.653853,73594.764875,245307.765249,247251.889219
2,1.594658e+07,5420.428917,238.727996,5738.046912,2735.767958,3753.105943,1387.434979,977.892985,1751.357973,15689.505760,...,39296.137399,38467.106412,16780.931743,16773.385743,4359.872933,4644.905929,52177.502202,53458.607183,168228.463427,169528.433408
3,1.588463e+07,4308.079934,133.769998,6529.004900,2982.041954,4208.609936,1649.486975,669.192990,1211.132981,15564.653762,...,49916.446237,48905.625252,19899.144696,20538.839686,4845.560926,4675.089929,56435.161137,55560.854150,170992.014385,169872.462402
4,1.590237e+07,31337.165521,1242.345981,6087.220907,3054.757953,5442.723917,1690.303974,2477.145962,1531.837977,17082.085739,...,41356.538368,36413.565443,17035.780739,15292.311766,4723.795928,5155.975921,47244.133278,48509.117258,175719.926313,169003.300416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,1.569422e+07,6103.684907,324.134995,7714.412882,3909.513940,4393.486933,1831.276972,708.294989,1659.433975,18643.078715,...,54671.798164,58074.358112,29851.632544,27525.749579,5071.254922,4902.155925,63139.782034,65053.379005,213351.484737,215086.035711
432,1.538854e+07,8439.857871,297.037995,9776.871850,4169.164936,5788.810911,2158.498967,799.189988,1459.464978,22824.591651,...,68433.643954,68585.935951,33448.330489,33660.647485,7181.733890,6543.410900,86576.628676,84725.114704,278761.240737,277187.899761
433,1.577001e+07,26875.078589,1254.350981,6626.073899,3760.994942,4147.555937,1796.976973,1557.562976,1391.207979,19399.393703,...,42258.971354,44293.647323,17078.998739,16321.654750,5251.329920,5586.783915,60745.299071,60598.495073,201174.298924,199671.958947
434,1.586539e+07,29671.214546,485.687993,5801.158911,2985.471954,3590.866945,1601.809976,1925.258971,1488.276977,17595.556731,...,41623.049363,43914.289328,17860.352727,17592.812731,5160.777921,5525.729915,52589.102196,51571.421211,186564.900147,183448.745195


In [51]:

brain_areas = set()
left_right_columns = []

for col in new_df1.columns:
    if col.startswith('Left-') or col.startswith('Right-'):
        left_right_columns.append(col)
        brain_area = col.split('-', 1)[1]
        brain_areas.add(brain_area)


for area in brain_areas:
    left_col = f'Left-{area}'
    right_col = f'Right-{area}'
    new_df1[area] = new_df1.get(left_col, 0) + new_df1.get(right_col, 0)

volume_df_final = new_df1.drop(columns=[col for col in left_right_columns if col not in ['subject']])

In [52]:
volume_df_final 

,Unknown,3rd-Ventricle,4th-Ventricle,Brain-Stem,CSF,WM-hypointensities,subject,Frontal,Amygdala,Combined-Cerebral-WM,...,Temporal,Parietal,Putamen,Occipital,Lateral-Ventricle,Accumbens-area,Cingulate,Insula,VentralDC,Cerebellum
0,1.584253e+07,2313.191965,3629.625944,16986.731740,1514.344977,21599.395670,OAS1_0299_MR1,133861.235953,2356.752964,348438.259672,...,84730.945704,84738.148704,7490.776885,37857.938421,61610.002058,694.231989,17642.890730,9666.425852,6214.130905,100203.332468
1,1.553299e+07,1145.962982,1914.625971,21888.544665,1116.121983,1429.966978,OAS1_0131_MR1,196288.949998,3950.330940,492559.654468,...,122743.234123,117944.321196,12221.775813,59727.275087,12684.139806,1574.026976,25908.161604,13323.834796,8596.265869,148569.418728
2,1.594658e+07,977.892985,1751.357973,15689.505760,1010.134985,2882.571956,OAS1_0355_MR1,129122.691025,2448.676963,337756.896835,...,80421.150770,77763.243811,7648.556883,33554.317487,11517.253824,817.711987,17757.452728,9004.778862,5907.488910,105636.109385
3,1.588463e+07,669.192990,1211.132981,15564.653762,840.006987,1099.657983,OAS1_0387_MR1,145096.200781,2981.012954,340864.476787,...,88020.658654,98822.071489,8377.431872,40437.984382,8692.305867,1102.058983,19028.610709,9520.650854,6479.612901,111996.015287
4,1.590237e+07,2477.145962,1531.837977,17082.085739,1187.122982,11633.187822,OAS1_0380_MR1,117483.672203,2499.097962,344723.226728,...,76100.036836,77770.103811,10541.075839,32328.092506,55950.159144,739.164989,17761.568728,9879.771849,6679.924898,95753.250536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,1.569422e+07,708.294989,1659.433975,18643.078715,1010.477985,1163.112982,OAS1_0104_MR1,169230.366412,2997.819954,428437.520448,...,102956.593426,112746.156276,8699.508867,57377.382123,12018.376816,1328.781980,21439.214672,9973.410847,7410.514887,128193.161040
432,1.538854e+07,799.189988,1459.464978,22824.591651,814.967988,2095.043968,OAS1_0108_MR1,217826.948669,3575.431945,555949.140498,...,128044.985042,137019.579905,11634.559822,67108.977974,13712.110790,1487.247977,26920.697588,13725.144790,8991.058863,171301.743380
433,1.577001e+07,1557.562976,1391.207979,19399.393703,1655.660975,9025.701862,OAS1_0425_MR1,141635.673834,2501.841962,400846.257870,...,82207.494743,86552.618676,8212.791874,33400.653489,63966.755022,1046.492984,18109.370723,10838.113834,6717.311897,121343.794144
434,1.586539e+07,1925.258971,1488.276977,17595.556731,1658.061975,3296.572950,OAS1_0035_MR1,126132.074071,1823.730972,370013.645342,...,77521.085815,85537.338692,7312.416888,35453.165458,51251.745216,914.780986,19158.950707,10686.507837,5929.097909,104160.523407


In [57]:
volume_df_final = volume_df_final.drop(columns = ['WM-hypointensities']) 

KeyError: "['WM-hypointensities'] not found in axis"

In [58]:
volume_df_final.columns

Index(['3rd-Ventricle', '4th-Ventricle', 'Brain-Stem', 'CSF', 'subject',
       'Frontal', 'Amygdala', 'Combined-Cerebral-WM', 'Caudate', 'Pallidum',
       'Inf-Lat-Vent', 'Hippocampus', 'Thalamus', 'Temporal', 'Parietal',
       'Putamen', 'Occipital', 'Lateral-Ventricle', 'Accumbens-area',
       'Cingulate', 'Insula', 'VentralDC', 'Cerebellum'],
      dtype='object')

In [59]:
volume_df_final.to_csv('all_oasis1_average_after_swift_fastsurfer_20240430_FFinal.csv')

In [60]:
pwd

'/home/limseoyoung/debug'